In [1]:
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import umap
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from tqdm import tqdm
datafile = "C:/Users/DEAL Lab/Desktop/서하린/새 폴더/pre_data/Moderna/moderna.txt"
documents = [line.strip() for line in open(datafile, encoding="cp949").readlines()]
preprocessed_documents = []

for line in tqdm(documents):
  # 빈 문자열이거나 숫자로만 이루어진 줄은 제외
  if line and not line.replace(' ', '').isdecimal():
    preprocessed_documents.append(line)
stop_word_file = "C:/Users/DEAL Lab/Desktop/서하린/새 폴더/covid19/txt/stopwords.txt"
stop_words =  [line.strip() for line in open(stop_word_file, encoding="utf-8").readlines()]

100%|████████████████████████████████████████████████████████████████████████| 30686/30686 [00:00<00:00, 930719.54it/s]


In [3]:
data = pd.DataFrame(preprocessed_documents)
data=data.drop_duplicates()

In [14]:
import re
from hanspell import spell_checker
docs_list = []
for temp in data[0]:
    temp=re.sub('[\{\}\[\]\/?.;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"\♥\♡\ㅋ\ㅠ\ㅜ\ㄱ\ㅎ\ㄲ\ㅡ]','', temp) #특수문자 제거
    #temp=re.sub('[a-zA-Z]' , '', temp) # 영어 제거
    temp=re.sub('[ㄱ-ㅣ]' , '', temp) # 자음모음 제거      
    #temp=re.sub('[0-9]' , '', temp) # 숫자제거
    result=spell_checker.check(temp)
    docs_list.append(result.checked)

In [15]:
docs_list

['아스트라제네카 모더나 화이자는 mrna 유전자 변형 내부 단백질 파괴 살인 백신입니다 실제 접종자의 접종 후기입니다',
 '일단 일본부터 가요 일본은 화이자 모더나 맞으니깐',
 '얀센은 기존 백신들에 비해 남아프리카 변이 바이러스에도 효과를 보이며 한번 접종만 해도 되며 그래서 더 저렴하며 유통에 있어서도 아스트라제네카처럼 용이하다고 한국은 화이자, 모더나 시판될 때 아직도 임상시험 중이었던 얀센을 미리 알아보고 선 구매한 것이 신의 한 수가 될 듯',
 '코로나 백신 구걸 물량 외에는 fda 승인받은 백신 0개의 굉장한 나라 딴 나라는 안전도 중하 l로 재고만 쌓이는 아스트라제네카도 부족 그리고 있어도 관리 못해서 진짜 랜덤박스 백신인 나라 가 아니 라로 뽑기 패가 바뀌었어',
 '프랑스 1일 확진자, 사망자 수 후들후들하고 오히려 반등 중 12월부터 백신 접종을 시작했음에도 접종률이 63 밖에 안 됨 화이자, 모더나, 아스트라제네카 다 있는데 왜 맞지를 못 하니 eu 국들아',
 ', 화이자 모더나 이어 얀센 세 번째 승인 문재인의 k 방역 백신은 언제 나오냐 한국이 백신 구입이 늦은 것은 k 방역의 백신이 곧 완성된다는 생각으로 미룬 것이다 전 세계가 만드는 백신을 k 방역은 물 건너 간 것이냐 k 방역이 백신을 만들었다면 난리 났다',
 '그래도 계약 완료 시점이 모더나 화이자가 10월 전인데 혼자 12월 10일이라고 된 거 보니 비즈니스 딜이 오고 가지 않았을까요  영국 일본 모두 good이고 말해도 good으로 알아먹으면 안 되는 나라들인데 ',
 'fda, 얀센 긴급 사용 승인 화이자 모더나 이어 세 번째',
 '아스트라제네카 75만 병 코로나 백신용 모더나 6만 병 이게 문재인이 구한 백신의 전부 나머지는 못 구함',
 '미국 질병통제예방센터는 생명을 위협하는 알레르기 반응인 아나필락시스는 100만 회 당 45건 수준으로 화이자 모더나 cdc는 독감 백신과 비슷한 수준이라고 밝혔다',
 '내 친구 모더나 맞았는데 같이 맞은 동료는 화요일에 

In [16]:
# Documents embeddings

model = SentenceTransformer('jhgan/ko-sroberta-multitask')
embeddings = model.encode(docs_list, show_progress_bar = True)

Batches:   0%|          | 0/958 [00:00<?, ?it/s]

In [17]:
# reduce dimension of embeddings

umap_embeddings = umap.UMAP(random_state = 1,n_neighbors =15, n_components =5, metric='cosine').fit_transform(embeddings)

In [18]:
import hdbscan
cluster = hdbscan.HDBSCAN(min_cluster_size=33,
                          metric='euclidean',                      
                          cluster_selection_method='eom').fit(umap_embeddings)

In [40]:
#from numpy import log as ln
#minpts = int(ln(len(umap_embeddings)))

In [41]:
#from sklearn.neighbors import NearestNeighbors
#import numpy as np
#neighbors = NearestNeighbors(n_neighbors=minpts)
#neighbors_fit = neighbors.fit(umap_embeddings)
#distances, indices = neighbors_fit.kneighbors(umap_embeddings)
#distances = np.sort(distances, axis=0)
#distances = distances[:,1]
#plt.plot(distances)

In [42]:
#clustering_model = DBSCAN(eps=0.6, min_samples = 15)
#cluster = clustering_model.fit(umap_embeddings)

In [19]:
cluster_label = cluster.labels_

In [20]:
df =[]
df = pd.DataFrame(docs_list)

In [21]:
df['label']=cluster_label
df['id']=range(len(df))
df.columns = ['docs','label','id']

In [22]:
df['label'].unique()

array([45, 27, 47, -1, 24, 22, 35, 38,  8, 36, 11, 42, 39, 25, 12, 17,  1,
       30, 40,  5, 43,  9, 15, 32, 28,  2, 29, 21, 41, 48, 37, 14, 20, 34,
       46, 10, 44, 33,  4, 23, 19, 26, 31, 18, 13,  6, 16,  0,  3,  7],
      dtype=int64)

In [73]:
docs_per_topic = df.groupby(['label'], as_index = False).agg({'docs': ' '.join})

In [25]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [76]:
for i in df[df['label']==45]['docs']:
    print(i)

아스트라제네카 모더나 화이자는 mrna 유전자 변형 내부 단백질 파괴 살인 백신입니다 실제 접종자의 접종 후기입니다
얀센은 기존 백신들에 비해 남아프리카 변이 바이러스에도 효과를 보이며 한번 접종만 해도 되며 그래서 더 저렴하며 유통에 있어서도 아스트라제네카처럼 용이하다고 한국은 화이자, 모더나 시판될 때 아직도 임상시험 중이었던 얀센을 미리 알아보고 선 구매한 것이 신의 한 수가 될 듯
lds 최소 잔여 형 주사기 코로나 백신, 아스트라제네카 아스트라제네카 화이자 모더나 풍림 파마텍
코로나 백신의 원리 인체의 b 세포가 바이러스와 싸우는데, 이 b 세포에게 바이러스 정보, 즉 유전자 단백질을 알려줘야 한다 1 바이러스 정보를 직접 주입한다 2 바이러스 정보를 만들게 초기 데이터를 주입한다 3 유전자재조합한 바이러스 정보를 주입한다
아스트라제네카 효과 60 바이러스 벡터의 유전자 주입 방식 화이자 모더나 효과 90 이상 mrna 주입 방식 노바백스 벡스 기존 단백질 주입 방식 효과 80fda 승인은 화이자 모더나만 받음 유전자 주입은 또 뭐야 mrna도 무서웠는데 유전자를 넣어
그리고 남아프리카 공화국은 그럼 아스트라제네카 빼고 화이자나 모더나 쓰나요 아니요 얀센의 얀센을 들여왔어요 아직 pda 승인 대기 중이고 효과는 66였지만 남아프리카공화국 변이 백신에도 57 이상의 효과를 보이며 상온 보관 가능하고 1회만 주사해도 된다고 합니다
개발 사별 코로나 백신 신뢰도 화이자 62, 모더나 53, 아스트라제네카 44 아스트라제네카 신뢰도, 정치 성향별 차이 큰 편 진보층 60, 중도보수층 40 내외
코로나 백신 원리 종류 특징 접종 시기 부작용 논란 평가 순위 k 주사기 아스트라제네카 화이자 모더나 얀센 노바백스
모더나, 화이자, 스푸트니크 v, 얀센 등 유효한 백신이 꾸준히 증가 중이다
아스트라제네카 모더나 화이자 rna 유전자 변형 살인 백신의 실체
한국에서 코로나 백신의 안전성에 대한 정치적 논란이 있는데요 참고로 영국에서 화이자, 아스트라제네카, 모더나 세

아스트라제네카 그리고 모더나 조합, 면역효과 최고 항체 17배 증가
노바백스 거는 단백질 기반 백신이라 기존의 모더나, 화이자의 mrna 기반 백신이랑 플랫폼이 다르죠 단백질 기반 백신은 신기술은 아니라 완전 신기술인 mrna 기반 백신보다는 안전성이 더 높을 거로 기대 중입니다
백신 교차접종 코로나 화이자 모더나도 화이자 화이자보다 효과 높아
영 연구팀 백신 7종 중 모더나 화이자 부스터 샷 효과 가장 커 이코노미 21 코로나 백신 부스터 샷 화이자 모더나 예방효과
란셋에 따르면 코로나 바이러스 백신 7종 중 모더나 화이자가 제일 좋아 코로나 바이러스 백신 부스터 샷
우리나라 하루 확진자 5000명 나오는 이유가 아스트라제네카 접종자 비 아스트라제네카 2차 접종 뒤 모더나 부스터 샷, 항체 32배 증가
아스트라제네카 2차 접종 뒤 모더나 부스터 샷, 항체 32배 증가
food  health 영국 연구진 아스트라제네카 2차 접종 뒤 모더나 부스터 샷, 항체 32배 증가
연구진 화이자 그리고 모더나 부스터 샷 교차접종 시 효과 1000 증가 부스터 샷 코로나 감염예방 효과 90 아스트라제네카 백신 그리고 화이자 항체 25배 증가 화이자 그리고 화이자 항체 8배 증가 연구진 아스트라제네카 혈전, 혈액 속 단백질과 백신 원소 결합 때문 아스트라제네카의 재발견
아스트라제네카화이자모더나얀센노바백스벡스발네바큐어백 추가접종 임상 결과 30세 이상 성인 2878명 대상 시험 안전성 문제 거의 없어
부스터 샷 어떤 백신이 가장 효과 좋나 모더나 화이자 우수
굉장히 실험 군을 넓게 잡아서, baselie은 아스트라제네카, 화이자 2차 접종 완료 군으로 설정 후 노바백스 half, 아스트라제네카, 얀센, 큐어 백, 발 네 발, 화이자 half, 모더나 등 사실상 모든 서구권 백신을 죄다 테스트했는데 큐어 백이랑 발 네 발, 아스트라제네카 그리고 얀센이 훌륭하진 않다, 정도 결과고 나머지 조합은 다 놀랍네요
화이자나 모던은 스파이크 단백질의 유전정보를 담은 전령 rna다  이 백신

In [60]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        sent = sent[:1000000]
        word_tokens = self.tagger.pos(sent,stem=True)
        tokens=[]
        #명사, 동사, 형용사만 추출
        for word in word_tokens:
            if word[1]=='Noun': #or word[1]=='Verb':
                tokens.append(word[0])
        result = [word for word in tokens if (len(word) > 1 
                                              and (not word in stop_words) )]
        #한 단어짜리 토큰 제거
        #불용어 제거
        return result

In [71]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Kkma,Mecab,Okt

def c_tf_idf(documents, m, tokenizer, ngram_range):
    count = CountVectorizer(ngram_range=ngram_range,tokenizer = tokenizer)
    count.fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count

def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
    words = count.get_feature_names()
    labels = list(docs_per_topic.label)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['label'])
                     .docs
                     .count()
                     .reset_index()
                     .rename({"label": "Topic", "docs": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return topic_sizes


In [74]:
custom_tokenizer = CustomTokenizer(Okt())
tf_idf, count = c_tf_idf(df.docs.values, len(df),custom_tokenizer,(1,1))
top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20)
topic_sizes = extract_topic_sizes(df)

<ipython-input-71-9f8e2e4395b6>:10: RuntimeWarning: invalid value encountered in true_divide
  tf = np.divide(t.T, w)


In [55]:
topic_sizes

,Topic,Size
18,17,28585
0,-1,630
4,3,294
15,14,249
17,16,168
9,8,165
12,11,134
11,10,97
10,9,68
3,2,57


In [61]:
custom_tokenizer = CustomTokenizer(Okt())

In [62]:
count_test = CountVectorizer(ngram_range=(1,1),tokenizer = custom_tokenizer)

In [63]:
count_test.fit(df.docs.values)

CountVectorizer(tokenizer=<__main__.CustomTokenizer object at 0x000002085CE534F0>)

In [64]:
count_test.vocabulary_

{'아스트라제네카': 9647,
 '모더나': 5134,
 '화이자': 17628,
 '유전자': 11733,
 '변형': 6439,
 '내부': 2536,
 '단백질': 3037,
 '파괴': 16112,
 '살인': 7610,
 '실제': 9338,
 '접종자': 13260,
 '일본': 12413,
 '얀센': 10065,
 '기존': 1999,
 '신들': 9211,
 '남아프리카': 2469,
 '변이': 6429,
 '바이러스': 5811,
 '효과': 17781,
 '보이': 6570,
 '한번': 16961,
 '해도': 17095,
 '유통': 11761,
 '한국': 16932,
 '시판': 9132,
 '임상시험': 12491,
 '미리': 5608,
 '구매': 1402,
 '신의': 9251,
 '수가': 8488,
 '구걸': 1366,
 '물량': 5495,
 '승인': 8990,
 '나라': 2316,
 '안전': 9827,
 '중하': 13982,
 '재고': 12932,
 '부족': 6813,
 '관리': 1222,
 '랜덤': 4169,
 '박스': 5870,
 '뽑기': 7319,
 '패가': 16242,
 '프랑스': 16623,
 '확진자': 17671,
 '사망자': 7409,
 '반등': 5914,
 '하니': 16811,
 '문재인': 5468,
 '방역': 6113,
 '구입': 1439,
 '완성': 11092,
 '세계': 8157,
 '건너': 609,
 '난리': 2416,
 '계약': 875,
 '시점': 9109,
 '비즈니스': 7165,
 '가지': 144,
 '영국': 10715,
 '긴급': 2037,
 '사용': 7455,
 '신용': 9246,
 '구한': 1462,
 '나머지': 2332,
 '구함': 1464,
 '미국': 5578,
 '질병통제예방센터': 14305,
 '생명': 7827,
 '위협': 11630,
 '알레르기': 9867,
 '반응': 5940,
 '아나필락시스': 95

In [75]:
top_n_words

{-1: [('파괴', 0.6928745182846185),
  ('내부', 0.6762998313033499),
  ('변형', 0.6464358252149831),
  ('살인', 0.5834224451640789),
  ('단백질', 0.5609806198975765),
  ('실제', 0.5330692530525846),
  ('유전자', 0.5092983712291521),
  ('접종자', 0.41495261813165174),
  ('아스트라제네카', 0.20125910918378978),
  ('화이자', 0.0999748182379921),
  ('힝힝', 0.0),
  ('발명자', 0.0),
  ('발목', 0.0),
  ('발뮤', 0.0),
  ('발미', 0.0),
  ('발병', 0.0),
  ('발바닥', 0.0),
  ('발발', 0.0),
  ('발명', 0.0),
  ('발뺌', 0.0)],
 0: [('일본', 2.017201950396684),
  ('화이자', 0.27493075015447827),
  ('힝힝', 0.0),
  ('발발', 0.0),
  ('발명자', 0.0),
  ('발목', 0.0),
  ('발뮤', 0.0),
  ('발미', 0.0),
  ('발바닥', 0.0),
  ('발병', 0.0),
  ('발송', 0.0),
  ('발뺌', 0.0),
  ('발사', 0.0),
  ('발산', 0.0),
  ('발상', 0.0),
  ('발생', 0.0),
  ('발설', 0.0),
  ('발명가', 0.0),
  ('발명', 0.0),
  ('발매', 0.0)],
 1: [('시판', 0.40651707005059656),
  ('신의', 0.3529418429441581),
  ('남아프리카', 0.3290285204033954),
  ('임상시험', 0.29171122258203946),
  ('보이', 0.28342387909140515),
  ('신들', 0.2797855741971535),
  (

In [130]:
from sklearn.metrics.pairwise import cosine_similarity
for i in range(20):
    # Calculate cosine similarity
    similarities = cosine_similarity(tf_idf.T)
    np.fill_diagonal(similarities, 0)

    # Extract label to merge into and from where
    topic_sizes = data.groupby(['label']).count().sort_values("docs", ascending=False).reset_index()
    topic_to_merge = topic_sizes.iloc[-1].label
    topic_to_merge_into = np.argmax(similarities[topic_to_merge + 1]) - 1

    # Adjust topics
    data.loc[data.label == topic_to_merge, "label"] = topic_to_merge_into
    old_topics = data.sort_values("label").label.unique()
    map_topics = {old_topic: index - 1 for index, old_topic in enumerate(old_topics)}
    data.label = data.label.map(map_topics)
    docs_per_topic = data.groupby(['label'], as_index = False).agg({'docs': ' '.join})

    # Calculate new topic words
    m = len(data)
    tf_idf, count = c_tf_idf(docs_per_topic.label.values, m)
    top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20)

topic_sizes = extract_topic_sizes(data); topic_sizes.head(10)

NameError: name 'c_tf_idf' is not defined